<a href="https://colab.research.google.com/github/ndow33/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
# TODO - code!

#1. Write a function

In [0]:
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):
  #This function returns the probability of a driver actually being a drunk driver
  #based on a breathalizer test

  #The numerator of the equation gives you the probability of a true positive 
  #multiplied by the overall probability of them being a drunk driver
  numerator = ((true_positive_rate - false_positive_rate)*prob_drunk_prior)

  #The denominator of the equation gives you the probability of a true positive
  #multiplied by the overall probability of them being a drunk driver. Then, we
  #we add the probability of a false positive occuring multiplied by 1 minus the 
  #overall probability of them being a drunk driver
  denominator = (((true_positive_rate-false_positive_rate)*(prob_drunk_prior))+
                 (false_positive_rate*(1-prob_drunk_prior)))
  return numerator/denominator

What is the probability that a person is drunk after one positive breathalyzer test?

In [2]:
prob_drunk_given_positive(1/1000, .08, 1)

0.011380504700643244

What is the probability that a person is drunk after two positive breathalyzer tests?

In [3]:
prior = prob_drunk_given_positive(1/1000, .08, 1)
prob_drunk_given_positive(prior, .08, 1)

0.11690607734806628

How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

In [6]:
prior = prob_drunk_given_positive(1/1000, .08, 1)
prior2 = prob_drunk_given_positive(prior, .08, 1)
prior3 = prob_drunk_given_positive(prior2, .08, 1)
prior4 = prob_drunk_given_positive(prior3, .08, 1)
prob_drunk_given_positive(prior4, .08, 1)

0.9950577515521439

It will take 5 breathalyzer tests in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit. 

#3) In your own words, summarize the difference between Bayesian and Frequentist statistics

The difference between Bayesian and Frequentist statistics is that Bayesian statistics uses updated probabilities to come up with new probabilities. Frequentist statistics is more static. Bayesian statistics considers the relationship between different variables and how they play into the probability of a certain event. 

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!

#I will find the probability of being a republican given the representative voted yes on a bill

In [0]:
import numpy as np
import pandas as pd
from scipy.stats import t, ttest_1samp, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-02-05 03:15:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-02-05 03:15:33 (612 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

In [13]:
df = pd.read_csv('house-votes-84.data', header = None,
                 names=column_headers,
                 na_values='?')

# recode votes as numeric
df = df.replace({'y':1, 'n':0})
df.head()

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [14]:
# how did Republicans vote?
rep = df[df['party']=='republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [21]:
rep.mean()

handicapped-infants     0.187879
water-project           0.506757
budget                  0.134146
physician-fee-freeze    0.987879
el-salvador-aid         0.951515
religious-groups        0.897590
anti-satellite-ban      0.240741
aid-to-contras          0.152866
mx-missile              0.115152
immigration             0.557576
synfuels                0.132075
education               0.870968
right-to-sue            0.860759
crime                   0.981366
duty-free               0.089744
south-africa            0.657534
dtype: float64

In [36]:
#this is the average support from republican representatives for any given bill
avg_rep_support = rep.mean()

avg_rep_support = avg_rep_support.mean()
avg_rep_support

0.5202842153559786

In [24]:
#this is the average support of bills amongst all representatives
df.mean()

handicapped-infants     0.442080
water-project           0.503876
budget                  0.596698
physician-fee-freeze    0.417453
el-salvador-aid         0.504762
religious-groups        0.641509
anti-satellite-ban      0.567696
aid-to-contras          0.576190
mx-missile              0.501211
immigration             0.504673
synfuels                0.362319
education               0.423267
right-to-sue            0.509756
crime                   0.593301
duty-free               0.427518
south-africa            0.812689
dtype: float64

In [38]:
avg_support = df.mean()
avg_support = avg_support.mean()
avg_support

0.5240624730030624

In [33]:
#This is the proportion of republican representatives
# how many from each party?
dems = df['party'].value_counts()[0]
reps = df['party'].value_counts()[1]

print(df['party'].value_counts().sort_index())
print(dems)
print(reps)

rep_prop = reps/(reps+dems)
rep_prop

democrat      267
republican    168
Name: party, dtype: int64
267
168


0.38620689655172413

In [0]:
#This function takes three arguments in this order: 
#1. the probability of a republican voting yes to a bill
#2. the probability of a republican being drawn at random 
#3. the probability of any of the representatives voting yes to a bill

def repub_given_yes(prob_repub_yes, prob_repub, prob_any_yes):
  numerator = prob_repub_yes*prob_repub
  denominator = prob_any_yes
  return numerator/denominator

In [39]:
repub_given_yes(avg_rep_support, rep_prop, avg_support)

0.38342251637679686

In other words, we could be 38% certain that the representative who was voting was a republican if they voted yes to one bill. What if they voted yes on two bills?

In [40]:
prior = repub_given_yes(avg_rep_support, rep_prop, avg_support)
repub_given_yes(avg_rep_support, prior, avg_support)

0.3806582103461372